<a href="https://colab.research.google.com/github/teruto725/predict_temperature/blob/master/seminer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##重回帰分析を用いて各データが最高気温に与える影響を明らかにする
#概要
データは気象庁のHPより奈良県の天気データを用いる。
最高気温を目標変数とし、降水量の合計、日照時間、平均風速、平均雲量を説明変数とする
#手法1
前処理として各データから平均値を引くことで中心化を行う

In [0]:
#driveとの連携
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/drive/My Drive/School Project/研究力向上セミナー


In [0]:
#ファイルの読み込み
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
df = pd.read_csv('data.csv', index_col=0)
df

,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),日照時間(時間),平均風速(m/s),平均雲量(10分比)
年月日,,,,,,,
2015/1/1,1.7,5.1,-1.1,2.0,4.0,2.6,7.7
2015/1/2,1.4,6.0,-1.5,0.0,1.9,2.6,9.7
2015/1/3,2.8,8.2,-0.9,0.0,4.8,1.7,3.7
2015/1/4,4.7,10.9,-1.4,0.0,4.6,1.6,7.7
2015/1/5,5.8,11.6,1.7,0.0,7.3,1.0,7.7
...,...,...,...,...,...,...,...
2018/12/28,3.1,7.3,-0.7,0.0,5.9,3.5,6.0
2018/12/29,2.6,7.5,-1.4,0.0,6.0,3.0,5.0
2018/12/30,2.5,7.5,-0.5,0.0,4.7,2.2,4.0


In [0]:
#目標変数と環境変数
t_df = df['最高気温(℃)']
x_df = DataFrame(df['平均雲量(10分比)']).combine_first(DataFrame(df['平均風速(m/s)'])).combine_first(DataFrame(df['日照時間(時間)'])).combine_first(DataFrame(df['降水量の合計(mm)']))
x_df

,平均雲量(10分比),平均風速(m/s),日照時間(時間),降水量の合計(mm)
年月日,,,,
2015/1/1,7.7,2.6,4.0,2.0
2015/1/2,9.7,2.6,1.9,0.0
2015/1/3,3.7,1.7,4.8,0.0
2015/1/4,7.7,1.6,4.6,0.0
2015/1/5,7.7,1.0,7.3,0.0
...,...,...,...,...
2018/12/28,6.0,3.5,5.9,0.0
2018/12/29,5.0,3.0,6.0,0.0
2018/12/30,4.0,2.2,4.7,0.0


In [0]:
#前処理
x_ave = x_df.mean()
x_df = x_df-x_ave


In [0]:
#モデルによる学習
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()
reg_model.fit(x_df,t_df)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#重み
reg_model.coef_


array([ 1.37447002, -1.03353168,  1.55149683,  0.11872549])

In [0]:
#バイアス（中心化したため0)
reg_model.intercept_

21.21203830369359

In [0]:
#決定係数
reg_model.score(x_df,t_df)

0.24378943357563376

決定係数があまりよくない

#手法2
前処理として標準化を行い手法1と比較する

In [0]:
#前処理（標準化)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_df)
x_df = scaler.transform(x_df)
x_df

array([[ 0.15936507,  1.21183827, -0.306425  , -0.17912017],
       [ 0.83903756,  1.21183827, -0.84679608, -0.35257516],
       [-1.19997991,  0.01814401, -0.10056936, -0.35257516],
       ...,
       [-1.09802903,  0.68130749, -0.12630131, -0.35257516],
       [ 0.70310306, -0.37975408, -1.20704347, -0.35257516],
       [-0.0785203 ,  0.1507767 ,  0.41406977, -0.30921141]])

In [0]:
#モデルによる学習
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()
reg_model.fit(x_df,t_df)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#重み
reg_model.coef_

array([ 4.04450687, -0.77924351,  6.02945546,  1.36894857])

In [0]:
#バイアス（標準化したため0)
reg_model.intercept_

21.21203830369357

In [0]:
#決定係数
reg_model.score(x_df,t_df)

0.24378943357563376

#結論
変化しなかった．

#手法3
説明変数に問題があると考え、新たな説明変数として合計全天日射量を追加

In [0]:
#決定係数
from pandas import Series,DataFrame
df = pd.read_csv('data_sunvalue.csv', index_col=0)
df

,最高気温(℃),降水量の合計(mm),平均雲量(10分比),日照時間(時間),合計全天日射量(MJ/㎡),平均風速(m/s)
年月日,,,,,,
2015/1/1,5.1,2.0,7.7,4.0,7.38,2.6
2015/1/2,6.0,0.0,9.7,1.9,6.50,2.6
2015/1/3,8.2,0.0,3.7,4.8,9.22,1.7
2015/1/4,10.9,0.0,7.7,4.6,8.26,1.6
2015/1/5,11.6,0.0,7.7,7.3,10.46,1.0
...,...,...,...,...,...,...
2018/12/28,7.3,0.0,6.0,5.9,10.44,3.5
2018/12/29,7.5,0.0,5.0,6.0,9.04,3.0
2018/12/30,7.5,0.0,4.0,4.7,8.04,2.2


In [0]:
#欠損のある行を削除
df = df.dropna()
df.isnull().sum()

最高気温(℃)          0
降水量の合計(mm)       0
平均雲量(10分比)       0
日照時間(時間)         0
合計全天日射量(MJ/㎡)    0
平均風速(m/s)        0
dtype: int64

In [0]:
#目標変数と説明変数に分割
t_df = DataFrame(df["最高気温(℃)"])
x_df = pd.concat([DataFrame(df["降水量の合計(mm)"]),DataFrame(df["日照時間(時間)"]),DataFrame(df['合計全天日射量(MJ/㎡)']),DataFrame(df['平均風速(m/s)']),DataFrame(df['平均雲量(10分比)'])],axis=1,join='inner')


In [0]:
#前処理
x_ave = x_df.mean()
x_df = x_df-x_ave
x_df

,降水量の合計(mm),日照時間(時間),合計全天日射量(MJ/㎡),平均風速(m/s),平均雲量(10分比)
年月日,,,,,
2015/1/1,-2.073681,-1.189102,-6.197019,0.914668,0.466415
2015/1/2,-4.073681,-3.289102,-7.077019,0.914668,2.466415
2015/1/3,-4.073681,-0.389102,-4.357019,0.014668,-3.533585
2015/1/4,-4.073681,-0.589102,-5.317019,-0.085332,0.466415
2015/1/5,-4.073681,2.110898,-3.117019,-0.685332,0.466415
...,...,...,...,...,...
2018/12/28,-4.073681,0.710898,-3.137019,1.814668,-1.233585
2018/12/29,-4.073681,0.810898,-4.537019,1.314668,-2.233585
2018/12/30,-4.073681,-0.489102,-5.537019,0.514668,-3.233585


In [0]:
#学習
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()
reg_model.fit(x_df,t_df)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#重み
reg_model.coef_

array([[ 0.14959149, -1.78370074,  1.69700284, -1.84055368,  0.32522739]])

In [0]:
#バイアス（標準化したため0)
reg_model.intercept_

array([21.2304318])

In [0]:
#決定係数
reg_model.score(x_df,t_df)

0.5952703859618937

#結果
なぜか日照時間の重みが負の値になってしまった。合計全日射量はいい感じ